In [1]:
import pandas as pd
import numpy as np
import holidays
from sklearn.preprocessing import LabelEncoder

In [ ]:
# NOTE: Custome Module !!
from module.dataframe import df_query_by_type, series_string_filter
from module.preprocess import do_preprocessing

---

In [4]:
train = pd.read_parquet("./database/train.parquet")
test = pd.read_parquet("./database/test.parquet")

In [5]:
train, test = do_preprocessing(train, test)

In [6]:
train_x, train_y = train.drop(columns="target"), train["target"]
test_x = test

---

In [7]:
import numpy as np
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor

from lightgbm import LGBMRegressor

In [8]:
x_train, x_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.2, random_state=42)
eval_set = [(x_valid, y_valid)]

linear_reg_model = LinearRegression()
decisiontree_reg_model = DecisionTreeRegressor()
randomforest_reg_model = RandomForestRegressor(n_estimators=50, random_state=42)
lgb_reg_model1 = LGBMRegressor(
    n_estimators=800,
    learning_rate=0.005,
    subsample=0.75,
    colsample_bytree=0.75,
    reg_alpha=0.1,
)
lgb_reg_model2 = LGBMRegressor(
    n_estimators=800,
    learning_rate=0.005,
    subsample=0.75,
    colsample_bytree=0.75,
    reg_alpha=0.1,
)

voting_ensemble_model = VotingRegressor(
    [
        ("linear", linear_reg_model),
        ("deicision_tree", decisiontree_reg_model),
        ("random_forest", randomforest_reg_model),
        ("lgbm1", lgb_reg_model1),
        ("lgbm2", lgb_reg_model2),
    ],
    n_jobs=16,
)

voting_ensemble_model.fit(x_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.154744 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 127
[LightGBM] [Info] Number of data points in the train set: 3760973, number of used features: 28
[LightGBM] [Info] Start training from score 42.786869
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.214416 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 127
[LightGBM] [Info] Number of data points in the train set: 3760973, number of used features: 28
[LightGBM] [Info] Start training from score 42.786869


VotingRegressor(estimators=[('linear', LinearRegression()),
                            ('deicision_tree', DecisionTreeRegressor()),
                            ('random_forest',
                             RandomForestRegressor(n_estimators=50,
                                                   random_state=42)),
                            ('lgbm1',
                             LGBMRegressor(colsample_bytree=0.75,
                                           learning_rate=0.005,
                                           n_estimators=800, reg_alpha=0.1,
                                           subsample=0.75)),
                            ('lgbm2',
                             LGBMRegressor(colsample_bytree=0.75,
                                           learning_rate=0.005,
                                           n_estimators=800, reg_alpha=0.1,
                                           subsample=0.75))],
                n_jobs=16)

In [13]:
import pickle
with open("./model/votingLinearLGB2.pkl","wb") as f:
    pickle.dump(voting_ensemble_model,f)

In [ ]:
# Eval
from sklearn.metrics import r2_score as R2
from sklearn.metrics import median_absolute_error as MAE

[
    MAE(voting_ensemble_model.predict(x_valid), y_valid),
    R2(voting_ensemble_model.predict(x_valid), y_valid),
]

[6.433551771081483, 0.21086316002712335, 10.17644136701328]

## Submission
---

In [15]:
sample_submission = pd.read_csv('./database/sample_submission.csv')

In [16]:
sample_submission["target"] = voting_ensemble_model.predict(test_x)
sample_submission.to_csv("./votingLinearLGB2.csv", index = False)
sample_submission

,id,target
0,TEST_000000,26.280760
1,TEST_000001,47.434525
2,TEST_000002,49.389008
3,TEST_000003,37.092323
4,TEST_000004,39.179237
...,...,...
291236,TEST_291236,50.790140
291237,TEST_291237,38.676616
291238,TEST_291238,20.430416
291239,TEST_291239,29.108617
